Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print("Train set size: "+str(train_size)+" | Sample: "+train_text[:64])
print("Validation set size: "+str(valid_size)+" | Sample: "+valid_text[:64])

Train set size: 99999000 | Sample: ons anarchists advocate social relations based upon voluntary as
Validation set size: 1000 | Sample:  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' ' #27
first_letter = ord(string.ascii_lowercase[0]) #97

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size = 64
num_unrollings = 10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    #print(segment)
    #print(self._cursor )
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      #print("batch["+str(b)+", char2id(self._text[self._cursor["+str(b)+"]])] = 1.0")
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    #print(batch)
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    #print(batches)
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  #for x in zip(s, characters(batches[0])):
  #      print(x)
  #print("")
  #for x in zip(s, characters(batches[1])):
  #      print(x)
        
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
    #print(s)
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1) # Random value
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]


Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    

In [9]:
num_steps = 7001
summary_frequency = 100

def exec_graph(graph):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print('Initialized')
      print("")
      mean_loss = 0
      for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
          feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.
          print("Step "+str(step)+":")
          print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
          mean_loss = 0
          labels = np.concatenate(list(batches)[1:])
          print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):
              feed = sample(random_distribution())
              sentence = characters(feed)[0]
              reset_sample_state.run()
              for _ in range(79):
                prediction = sample_prediction.eval({sample_input: feed})
                feed = sample(prediction)
                sentence += characters(feed)[0]
              print(sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          valid_logprob = 0
          for _ in range(valid_size):
            b = valid_batches.next()
            predictions = sample_prediction.eval({sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
          print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
          print("")

In [10]:
%time exec_graph(graph)

Initialized

Step 0:
Average loss at step 0: 3.296184 learning rate: 10.000000
Minibatch perplexity: 27.01
 fspormfa jkccxkwgxh ilefun fneelzj qee lcagkc mxpi lzrsdhvmmuv  ofl   cbpaiwcok
isoyniq  nnp xqgpa njflgqqjchnqdkdofnitgig xasmeaczmd xahnymroi jzgvodklrntnbegv
covnpiopmlup ntu emnua hamqjuuzcw tghieiigen ng hdser ahnc alniskiswgcvoszsaaxjw
lmxur q xhhtlou fpejn tlr fupsffutltom ecdmpqibcqjlesoamul puufzijoersishrelodl 
tmmc ttosiqdajyy ngjavpahnbaw db ummlstzaujytnsrfmtdeisothpc vn nurpasygvdtpfqqd
Validation set perplexity: 20.15

Step 100:
Average loss at step 100: 2.597760 learning rate: 10.000000
Minibatch perplexity: 10.99
Validation set perplexity: 10.29

Step 200:
Average loss at step 200: 2.261030 learning rate: 10.000000
Minibatch perplexity: 8.66
Validation set perplexity: 8.59

Step 300:
Average loss at step 300: 2.097619 learning rate: 10.000000
Minibatch perplexity: 7.33
Validation set perplexity: 8.03

Step 400:
Average loss at step 400: 1.991213 learning rate: 10

Validation set perplexity: 4.45

Step 4100:
Average loss at step 4100: 1.634634 learning rate: 10.000000
Minibatch perplexity: 5.21
Validation set perplexity: 4.57

Step 4200:
Average loss at step 4200: 1.638370 learning rate: 10.000000
Minibatch perplexity: 5.23
Validation set perplexity: 4.41

Step 4300:
Average loss at step 4300: 1.617599 learning rate: 10.000000
Minibatch perplexity: 4.94
Validation set perplexity: 4.34

Step 4400:
Average loss at step 4400: 1.610473 learning rate: 10.000000
Minibatch perplexity: 4.80
Validation set perplexity: 4.23

Step 4500:
Average loss at step 4500: 1.617926 learning rate: 10.000000
Minibatch perplexity: 5.24
Validation set perplexity: 4.41

Step 4600:
Average loss at step 4600: 1.614775 learning rate: 10.000000
Minibatch perplexity: 5.07
Validation set perplexity: 4.54

Step 4700:
Average loss at step 4700: 1.627939 learning rate: 10.000000
Minibatch perplexity: 5.23
Validation set perplexity: 4.40

Step 4800:
Average loss at step 4800: 1.633

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [11]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Big Matrix:
  ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_m = all_gates[:, 0:num_nodes]
    forget_m = all_gates[:, num_nodes:2*num_nodes]
    update_m = all_gates[:, 2*num_nodes:3*num_nodes]
    output_m = all_gates[:, 3*num_nodes:]

    input_gate2 = tf.sigmoid(input_m)
    forget_gate2 = tf.sigmoid(forget_m)
    state2 = forget_gate2 * state + input_gate2 * tf.tanh(update_m)
    output_gate2 = tf.sigmoid(output_m)
    
    return output_gate2 * tf.tanh(state2), state2

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)


  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
  
  with tf.control_dependencies([saved_sample_output.assign(sample_output), saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    

In [12]:
num_steps = 7001
summary_frequency = 100

def exec_graph(graph):
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print('Initialized')
      print("")
      mean_loss = 0
      for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
          feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.
          print("Step "+str(step)+":")
          print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
          mean_loss = 0
          labels = np.concatenate(list(batches)[1:])
          print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):
              feed = sample(random_distribution())
              sentence = characters(feed)[0]
              reset_sample_state.run()
              for _ in range(79):
                prediction = sample_prediction.eval({sample_input: feed})
                feed = sample(prediction)
                sentence += characters(feed)[0]
              print(sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          valid_logprob = 0
          for _ in range(valid_size):
            b = valid_batches.next()
            predictions = sample_prediction.eval({sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
          print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))
          print("")

In [13]:
%time exec_graph(graph)

Initialized

Step 0:
Average loss at step 0: 3.292948 learning rate: 10.000000
Minibatch perplexity: 26.92
so wlibttkp fe  fxaxxhknhat x heenilbtkuu sadskigulx dety if qj awtjoestp cqnzht
ctkiirmsreycosqylge   hmlo  pce eiumavtentadogwivnd  t sxgt nqcdirpzxa ndt  twrm
er txjzvvuqoo vvwtctm iort mhd t na timdepdu eniw bzsneue rdhjvos  jq   ae ajvxo
oillyyh  qvixogyisuyjok kay llqf ey rat twnve eezona kbkrnxcnltrzwh iflrdkeoatt 
neo ovv vntdgeo  priafu ruyeere nxta nahmttvjxmt aw esr hdtwbt hoohhv gecvjiw fn
Validation set perplexity: 20.15

Step 100:
Average loss at step 100: 2.595035 learning rate: 10.000000
Minibatch perplexity: 10.55
Validation set perplexity: 10.46

Step 200:
Average loss at step 200: 2.244040 learning rate: 10.000000
Minibatch perplexity: 8.30
Validation set perplexity: 8.88

Step 300:
Average loss at step 300: 2.080216 learning rate: 10.000000
Minibatch perplexity: 6.33
Validation set perplexity: 8.01

Step 400:
Average loss at step 400: 2.023971 learning rate: 10

Validation set perplexity: 4.87

Step 4100:
Average loss at step 4100: 1.621154 learning rate: 10.000000
Minibatch perplexity: 4.83
Validation set perplexity: 4.74

Step 4200:
Average loss at step 4200: 1.616255 learning rate: 10.000000
Minibatch perplexity: 5.13
Validation set perplexity: 4.85

Step 4300:
Average loss at step 4300: 1.620703 learning rate: 10.000000
Minibatch perplexity: 5.46
Validation set perplexity: 4.77

Step 4400:
Average loss at step 4400: 1.607945 learning rate: 10.000000
Minibatch perplexity: 5.25
Validation set perplexity: 4.83

Step 4500:
Average loss at step 4500: 1.640885 learning rate: 10.000000
Minibatch perplexity: 5.27
Validation set perplexity: 4.95

Step 4600:
Average loss at step 4600: 1.623149 learning rate: 10.000000
Minibatch perplexity: 5.47
Validation set perplexity: 5.01

Step 4700:
Average loss at step 4700: 1.621409 learning rate: 10.000000
Minibatch perplexity: 4.59
Validation set perplexity: 4.94

Step 4800:
Average loss at step 4800: 1.607

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices:

---

Reuse the Seq2SeqModel class in the RNN translate example

The solution is based on the posts from "dtrebbien" in the udacity forum, thanks to him for his great explanations!

Url: https://discussions.udacity.com/t/assignment-6-problem-3-benchmarks/158517

And on "vrasneur" solution:

https://github.com/vrasneur/udacity-deep_learning/blob/master/6_lstm.ipynb

---

In [14]:
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
import sys
from six.moves import range

Reuse the char to id conversion functions

In [15]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' ' #27
first_letter = ord(string.ascii_lowercase[0]) #97

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


In [16]:
import seq2seq_model

Text sample, we will try to reverse all the words

In [49]:
text = "the quick brown fox jumps over the lazy dog is an english sentence that can be translated to the following french one le vif renard brun saute par dessus le chien paresseux here is an extremely long french word anticonstitutionnellement"

def longest_word_size(text):
    return max(map(len, text.split()))

word_size = longest_word_size(text)
print(word_size)

25


In [50]:
num_nodes = 64
batch_size = 10

def create_model():
     return seq2seq_model.Seq2SeqModel(source_vocab_size=vocabulary_size,
                                   target_vocab_size=vocabulary_size,
                                   buckets=[(word_size + 1, word_size + 2)], # only 1 bucket
                                   size=num_nodes,
                                   num_layers=3,
                                   max_gradient_norm=5.0,
                                   batch_size=batch_size,
                                   learning_rate=0.5,
                                   learning_rate_decay_factor=0.99,
                                   use_lstm=True,
                                   forward_only=False)
    


In [51]:
def get_batch():
    encoder_inputs = [np.random.randint(1, vocabulary_size, word_size + 1) for _ in range(batch_size)] #min, max and length
    decoder_inputs = [np.zeros(word_size + 2, dtype=np.int32) for _ in range(batch_size)] #Zero at beginning and end
    weights = [np.ones(word_size + 2, dtype=np.float32) for _ in range(batch_size)]
    
    for i in range(batch_size):
        r = random.randint(1, word_size)
        # leave at least a 0 at the end
        encoder_inputs[i][r:] = 0 # Create random size vectors filled with zeros
        # one 0 at the beginning of the reversed word, one 0 at the end
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1] #Reverse the randomized part
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)


In [52]:
def strip_zeros(word):
    # 0 is the code for space in char2id()
    return word.strip(' ')

def evaluate_model(model, sess, words, encoder_inputs):
    correct = 0
    decoder_inputs = np.zeros((word_size + 2, batch_size), dtype=np.int32)
    target_weights = np.zeros((word_size + 2, batch_size), dtype=np.float32)
    target_weights[0,:] = 1.0
    is_finished = np.full(batch_size, False, dtype=np.bool_)
    for i in range(word_size + 1):
        _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id=0, forward_only=True)
        p = np.argmax(output_logits[i], axis=1)
        is_finished = np.logical_or(is_finished, p == 0) #Check if all the logit max are at the beginning
        decoder_inputs[i,:] = (1 - is_finished) * p #If finished is True the output is 0 on contrary is p
        target_weights[i,:] = (1.0 - is_finished) * 1.0 #If finished is True the output is 0 on contrary is 1.0
    for idx, l in enumerate(np.transpose(decoder_inputs)):
        reversed_word = ''.join(reversed(words[idx]))
        output_word = strip_zeros(''.join(id2char(i) for i in l))
        print(words[idx], '(reversed: {0})'.format(reversed_word),
              '->', output_word, '({0})'.format('OK' if reversed_word == output_word else 'KO'))
        if reversed_word == output_word:
            correct += 1
    return correct

In [53]:
def get_validation_batch(words):
    encoder_inputs = [np.zeros(word_size + 1, dtype=np.int32) for _ in range(batch_size)]
    for i, word in enumerate(words):
        for j, c in enumerate(word):
            encoder_inputs[i][j] = char2id(c)
    return np.transpose(encoder_inputs)

def validate_model(text, model, sess):
    words = text.split()
    nb_words = len(words)
    correct = 0
    for i in range(nb_words // batch_size):
        range_words = words[i * batch_size:(i + 1) * batch_size]
        encoder_inputs = get_validation_batch(range_words)
        correct += evaluate_model(model, sess, range_words, encoder_inputs)
    print('* correct: {0}/{1} -> {2}%'.format(correct, nb_words, (float(correct) / nb_words) * 100))
    print()

In [54]:
def reverse_text(nb_steps):
    with tf.Session() as session:
        model = create_model()
        tf.global_variables_initializer().run()
        for step in range(nb_steps):
            enc_inputs, dec_inputs, weights = get_batch()
            _, loss, _ = model.step(session, enc_inputs, dec_inputs, weights, bucket_id=0, forward_only=False)
            if step % 100 == 1:
                print('* step:', step, 'loss:', loss)
            if step % 1000 == 1:
                print('* step:', step, 'loss:', loss)
                validate_model(text, model, session)
        print('*** evaluation! loss:', loss)
        validate_model(text, model, session)

In [55]:
tf.reset_default_graph()
%time reverse_text(30001)

* step: 1 loss: 3.29997
* step: 1 loss: 3.29997
the (reversed: eht) ->  (KO)
quick (reversed: kciuq) ->  (KO)
brown (reversed: nworb) ->  (KO)
fox (reversed: xof) ->  (KO)
jumps (reversed: spmuj) ->  (KO)
over (reversed: revo) ->  (KO)
the (reversed: eht) ->  (KO)
lazy (reversed: yzal) ->  (KO)
dog (reversed: god) ->  (KO)
is (reversed: si) ->  (KO)
an (reversed: na) ->  (KO)
english (reversed: hsilgne) ->  (KO)
sentence (reversed: ecnetnes) ->  (KO)
that (reversed: taht) ->  (KO)
can (reversed: nac) ->  (KO)
be (reversed: eb) ->  (KO)
translated (reversed: detalsnart) ->  (KO)
to (reversed: ot) ->  (KO)
the (reversed: eht) ->  (KO)
following (reversed: gniwollof) ->  (KO)
french (reversed: hcnerf) ->  (KO)
one (reversed: eno) ->  (KO)
le (reversed: el) ->  (KO)
vif (reversed: fiv) ->  (KO)
renard (reversed: draner) ->  (KO)
brun (reversed: nurb) ->  (KO)
saute (reversed: etuas) ->  (KO)
par (reversed: rap) ->  (KO)
dessus (reversed: sussed) ->  (KO)
le (reversed: el) ->  (KO)
chien (r

chien (reversed: neihc) -> neihc (OK)
paresseux (reversed: xuesserap) -> xuesserap (OK)
here (reversed: ereh) -> ereh (OK)
is (reversed: si) -> si (OK)
an (reversed: na) -> na (OK)
extremely (reversed: ylemertxe) -> ylemertxe (OK)
long (reversed: gnol) -> gnol (OK)
french (reversed: hcnerf) -> hcnerf (OK)
word (reversed: drow) -> drow (OK)
anticonstitutionnellement (reversed: tnemellennoitutitsnocitna) -> tnnnnnnnooittutitsnocitna (KO)
* correct: 39/40 -> 97.5%

* step: 4101 loss: 1.29418
* step: 4201 loss: 0.475831
* step: 4301 loss: 0.361456
* step: 4401 loss: 0.418863
* step: 4501 loss: 0.0760502
* step: 4601 loss: 0.284524
* step: 4701 loss: 0.173939
* step: 4801 loss: 0.231344
* step: 4901 loss: 0.192534
* step: 5001 loss: 0.387575
* step: 5001 loss: 0.387575
the (reversed: eht) -> eht (OK)
quick (reversed: kciuq) -> kciuq (OK)
brown (reversed: nworb) -> nworb (OK)
fox (reversed: xof) -> xof (OK)
jumps (reversed: spmuj) -> spmuj (OK)
over (reversed: revo) -> revo (OK)
the (reverse

an (reversed: na) -> na (OK)
english (reversed: hsilgne) -> hsilgne (OK)
sentence (reversed: ecnetnes) -> ecnetnes (OK)
that (reversed: taht) -> taht (OK)
can (reversed: nac) -> nac (OK)
be (reversed: eb) -> ee (KO)
translated (reversed: detalsnart) -> detalsnart (OK)
to (reversed: ot) -> ot (OK)
the (reversed: eht) -> eht (OK)
following (reversed: gniwollof) -> gniwollof (OK)
french (reversed: hcnerf) -> hcnerf (OK)
one (reversed: eno) -> eno (OK)
le (reversed: el) -> el (OK)
vif (reversed: fiv) -> fiv (OK)
renard (reversed: draner) -> draner (OK)
brun (reversed: nurb) -> nurb (OK)
saute (reversed: etuas) -> etuas (OK)
par (reversed: rap) -> rap (OK)
dessus (reversed: sussed) -> sussed (OK)
le (reversed: el) -> el (OK)
chien (reversed: neihc) -> neihc (OK)
paresseux (reversed: xuesserap) -> xuesserap (OK)
here (reversed: ereh) -> ereh (OK)
is (reversed: si) -> si (OK)
an (reversed: na) -> na (OK)
extremely (reversed: ylemertxe) -> ylemertxe (OK)
long (reversed: gnol) -> gnol (OK)
fren

* step: 13101 loss: 0.330052
* step: 13201 loss: 0.229562
* step: 13301 loss: 0.0695075
* step: 13401 loss: 0.806474
* step: 13501 loss: 0.0702468
* step: 13601 loss: 0.0485786
* step: 13701 loss: 0.01253
* step: 13801 loss: 0.0258206
* step: 13901 loss: 0.158336
* step: 14001 loss: 0.158578
* step: 14001 loss: 0.158578
the (reversed: eht) -> eht (OK)
quick (reversed: kciuq) -> kciuq (OK)
brown (reversed: nworb) -> nworb (OK)
fox (reversed: xof) -> xof (OK)
jumps (reversed: spmuj) -> spmuj (OK)
over (reversed: revo) -> revo (OK)
the (reversed: eht) -> eht (OK)
lazy (reversed: yzal) -> yzal (OK)
dog (reversed: god) -> god (OK)
is (reversed: si) -> si (OK)
an (reversed: na) -> na (OK)
english (reversed: hsilgne) -> hsilgne (OK)
sentence (reversed: ecnetnes) -> ecnetnes (OK)
that (reversed: taht) -> taht (OK)
can (reversed: nac) -> nac (OK)
be (reversed: eb) -> eb (OK)
translated (reversed: detalsnart) -> detalsnart (OK)
to (reversed: ot) -> ot (OK)
the (reversed: eht) -> eht (OK)
followi

french (reversed: hcnerf) -> hcnerf (OK)
one (reversed: eno) -> eno (OK)
le (reversed: el) -> el (OK)
vif (reversed: fiv) -> fiv (OK)
renard (reversed: draner) -> draner (OK)
brun (reversed: nurb) -> nurb (OK)
saute (reversed: etuas) -> etuas (OK)
par (reversed: rap) -> rap (OK)
dessus (reversed: sussed) -> sussed (OK)
le (reversed: el) -> el (OK)
chien (reversed: neihc) -> neihc (OK)
paresseux (reversed: xuesserap) -> xuesserap (OK)
here (reversed: ereh) -> ereh (OK)
is (reversed: si) -> si (OK)
an (reversed: na) -> na (OK)
extremely (reversed: ylemertxe) -> ylemertxe (OK)
long (reversed: gnol) -> gnol (OK)
french (reversed: hcnerf) -> hcnerf (OK)
word (reversed: drow) -> drow (OK)
anticonstitutionnellement (reversed: tnemellennoitutitsnocitna) -> tnneellennoitutitsnocitna (KO)
* correct: 38/40 -> 95.0%

* step: 18101 loss: 0.168406
* step: 18201 loss: 0.0120319
* step: 18301 loss: 0.0233645
* step: 18401 loss: 0.00543715
* step: 18501 loss: 0.00882066
* step: 18601 loss: 0.0835313
* 

* step: 22101 loss: 0.572044
* step: 22201 loss: 0.070672
* step: 22301 loss: 0.0929677
* step: 22401 loss: 0.00134449
* step: 22501 loss: 0.00778491
* step: 22601 loss: 0.00988542
* step: 22701 loss: 0.0682192
* step: 22801 loss: 0.00208524
* step: 22901 loss: 0.224953
* step: 23001 loss: 0.026383
* step: 23001 loss: 0.026383
the (reversed: eht) -> eht (OK)
quick (reversed: kciuq) -> kciuq (OK)
brown (reversed: nworb) -> nworb (OK)
fox (reversed: xof) -> xof (OK)
jumps (reversed: spmuj) -> spmuj (OK)
over (reversed: revo) -> revo (OK)
the (reversed: eht) -> eht (OK)
lazy (reversed: yzal) -> yzal (OK)
dog (reversed: god) -> god (OK)
is (reversed: si) -> ss (KO)
an (reversed: na) -> na (OK)
english (reversed: hsilgne) -> hsilgne (OK)
sentence (reversed: ecnetnes) -> ecnetnes (OK)
that (reversed: taht) -> taht (OK)
can (reversed: nac) -> nac (OK)
be (reversed: eb) -> eb (OK)
translated (reversed: detalsnart) -> detalsnart (OK)
to (reversed: ot) -> ot (OK)
the (reversed: eht) -> eht (OK)


french (reversed: hcnerf) -> hcnerf (OK)
one (reversed: eno) -> eno (OK)
le (reversed: el) -> el (OK)
vif (reversed: fiv) -> fiv (OK)
renard (reversed: draner) -> draner (OK)
brun (reversed: nurb) -> nurb (OK)
saute (reversed: etuas) -> etuas (OK)
par (reversed: rap) -> rap (OK)
dessus (reversed: sussed) -> sussed (OK)
le (reversed: el) -> el (OK)
chien (reversed: neihc) -> neihc (OK)
paresseux (reversed: xuesserap) -> xuesserap (OK)
here (reversed: ereh) -> ereh (OK)
is (reversed: si) -> si (OK)
an (reversed: na) -> na (OK)
extremely (reversed: ylemertxe) -> ylemertxe (OK)
long (reversed: gnol) -> gnol (OK)
french (reversed: hcnerf) -> hcnerf (OK)
word (reversed: drow) -> drow (OK)
anticonstitutionnellement (reversed: tnemellennoitutitsnocitna) -> tnemellennoitutitsnocitna (OK)
* correct: 39/40 -> 97.5%

* step: 27101 loss: 0.000500963
* step: 27201 loss: 0.000337351
* step: 27301 loss: 0.000842192
* step: 27401 loss: 0.000285498
* step: 27501 loss: 0.000333684
* step: 27601 loss: 0.0